In [ ]:
import re
import unicodedata
import hashlib
import pandas as pd
import numpy as np

In [ ]:
# Chargement des données brutes
df_m = pd.read_csv('Sneakers_hommes_decath.csv', sep=None, engine='python', encoding='utf-8-sig', on_bad_lines='skip')
df_f = pd.read_csv('Sneakers_femmes_decath.csv', sep=None, engine='python', encoding='utf-8-sig', on_bad_lines='skip')
df_c = pd.read_csv('Sneakers_enfants_decath.csv', sep=None, engine='python', encoding='utf-8-sig', on_bad_lines='skip')

In [ ]:
# vérification des données
print("Femmes:", df_f.shape)
print("Hommes:", df_m.shape)
print("Enfants:", df_c.shape)

Femmes: (1185, 10)
Hommes: (1239, 10)
Enfants: (419, 10)


In [ ]:
# Vérification des valeurs manquantes dans le fichier 'hommes'
print("--- Valeurs manquantes dans Sneakers_hommes_decath.csv ---")
print(df_m.isnull().sum())

print("\n" + "="*50 + "\n")

# Vérification des valeurs manquantes dans le fichier 'femmes'
print("--- Valeurs manquantes dans Sneakers_femmes_decath.csv ---")
print(df_f.isnull().sum())

print("\n" + "="*50 + "\n")

# Vérification des valeurs manquantes dans le fichier 'enfants'
print("--- Valeurs manquantes dans Sneakers_enfants_decath.csv ---")
print(df_c.isnull().sum())

--- Valeurs manquantes dans Sneakers_hommes_decath.csv ---
Prix_actuel                  267
Prix_initial                 963
Baisse_du_prix               963
Durée_de_vente               963
Marque                         1
Description                    1
Commentaire                 1059
Livraison                   1140
Image_URL                      1
URL_de_la_page_détaillée       1
dtype: int64


--- Valeurs manquantes dans Sneakers_femmes_decath.csv ---
Prix_actuel                  249
Prix_initial                 915
Baisse_du_prix               915
Durée_de_vente               915
Marque                         1
Description                    1
Commentaire                 1016
Livraison                   1085
Image_URL                      1
URL_de_la_page_détaillée       1
dtype: int64


--- Valeurs manquantes dans Sneakers_enfants_decath.csv ---
Prix_actuel                 106
Prix_initial                282
Baisse_du_prix              282
Durée_de_vente              282
Marq

In [ ]:
# Fonction pour nettoyer et vérifier un fichier CSV
# exclusion des marques.
exclude_brands = [
    "Quechua", "Salomon", "Merrell", "Garmont", "Grisport", "La Sportiva",
    "Mammut", "Salewa", "Jack Wolfskin", "Helly Hansen", "Columbia",
    "Clarks", "Panama Jack", "Birkenstock", "Aigle", "Brooks", "Ugg"
]
exclude_keywords = ["sandales", "ballerines", "bottes", "crampons", "randonnée"]

def clean_and_load_csv(filename, gender):
    try:
        df = pd.read_csv(filename, sep=None, engine='python', encoding='utf-8-sig', on_bad_lines='skip')
        df.columns = [col.strip().replace(' ', '_').replace('€', '') for col in df.columns]
        df = df.rename(columns={'Prix_actuel': 'price', 'Image_URL': 'image_url'})
        print(f"\n--- Nettoyage de {filename} ({gender}) ---")
        print(f"Lignes initiales : {len(df)}")

        # 1. Suppression des lignes avec valeurs manquantes
        df.dropna(subset=['price', 'image_url', 'Marque', 'Description'], inplace=True)
        print(f"Lignes après la suppression des valeurs manquantes : {len(df)}")

        # Normaliser les colonnes pour le filtrage
        df['Marque'] = df['Marque'].fillna('').str.strip()
        df['Description_lower'] = df['Description'].str.lower().fillna('')

        # 2. **Filtrage des marques et des mots-clés (comme demandé)**
        df = df[~df['Marque'].str.upper().isin([b.upper() for b in exclude_brands])]
        df = df[~df['Description_lower'].str.contains('|'.join(exclude_keywords))]

        # Supprimer la colonne temporaire
        df = df.drop(columns=['Description_lower'])

        print(f"Lignes après le filtrage des marques et mots-clés : {len(df)}")
        print("Vérification après nettoyage :")
        print(df[['price', 'image_url', 'Marque', 'Description']].isnull().sum())

        df['gender'] = gender

        return df
    except FileNotFoundError:
        print(f"Erreur : Le fichier {filename} n'a pas été trouvé.")
        return None


In [ ]:
# Chargement et nettoyage de chaque fichier
df_m = clean_and_load_csv('Sneakers_hommes_decath.csv', 'men')
df_f = clean_and_load_csv('Sneakers_femmes_decath.csv', 'women')
df_c = clean_and_load_csv('Sneakers_enfants_decath.csv', 'child')

# Concaténation des DataFrames nettoyés
df_list = [df for df in [df_m, df_f, df_c] if df is not None]
df_concat = pd.concat(df_list, ignore_index=True)



--- Nettoyage de Sneakers_hommes_decath.csv (men) ---
Lignes initiales : 1239
Lignes après la suppression des valeurs manquantes : 972
Lignes après le filtrage des marques et mots-clés : 941
Vérification après nettoyage :
price          0
image_url      0
Marque         0
Description    0
dtype: int64

--- Nettoyage de Sneakers_femmes_decath.csv (women) ---
Lignes initiales : 1185
Lignes après la suppression des valeurs manquantes : 936
Lignes après le filtrage des marques et mots-clés : 912
Vérification après nettoyage :
price          0
image_url      0
Marque         0
Description    0
dtype: int64

--- Nettoyage de Sneakers_enfants_decath.csv (child) ---
Lignes initiales : 419
Lignes après la suppression des valeurs manquantes : 313
Lignes après le filtrage des marques et mots-clés : 267
Vérification après nettoyage :
price          0
image_url      0
Marque         0
Description    0
dtype: int64


In [ ]:
# Affichage des premières lignes pour vérification
print(df_concat.head())

    price Prix_initial Baisse_du_prix  \
0  39,99€       64,99€        -25,00€   
1  12,99€          NaN            NaN   
2     23€          NaN            NaN   
3  24,99€       34,99€        -10,00€   
4  89,99€          NaN            NaN   

                                     Durée_de_vente     Marque  \
0  *À partir du 06/08/2025 et avant le 30/09/2025       ADIDAS   
1                                               NaN  DECATHLON   
2                                               NaN     TARMAK   
3                    *Jusqu'à épuisement du stock    DECATHLON   
4                                               NaN     KIPSTA   

                                         Description Commentaire  \
0         CHAUSSURE HOMME ULTIMASHOW 2.0 ADIDAS NOIR       (159)   
1                   Chaussures homme Ekiden One gris     (10742)   
2  Chaussures de basketball homme/femme - SE 500 ...       (424)   
3       Baskets en cuir CJ80 homme blanc et bordeaux       (646)   
4  Chaussure de 

In [ ]:
# Normalisation des prix et des marques
df_concat['price'] = df_concat['price'].astype(str).str.replace('€', '').str.replace(',', '.').str.strip()
df_concat = df_concat[pd.to_numeric(df_concat['price'], errors='coerce').notnull()]
df_concat['price'] = df_concat['price'].astype(float)
df_concat['Marque'] = df_concat['Marque'].fillna('').str.upper().str.strip()

# Filtrage
exclude_brands = [
    "Quechua", "Salomon", "Merrell", "Garmont", "Grisport", "La Sportiva",
    "Mammut", "Salewa", "Jack Wolfskin", "Helly Hansen", "Columbia",
    "Clarks", "Panama Jack", "Birkenstock", "Aigle", "Brooks", "Ugg"
]
exclude_keywords = ["sandales", "ballerines", "bottes", "crampons", "randonnée"]
df_concat = df_concat[~df_concat['Marque'].isin(exclude_brands)]
df_concat = df_concat[~df_concat['Description'].str.lower().str.contains('|'.join(exclude_keywords))]

print(f"✅ OK: Données chargées et nettoyées. Lignes finales : {len(df_concat)}")

✅ OK: Données chargées et nettoyées. Lignes finales : 2120


In [ ]:
# --- Fonctions et listes de nettoyage du nom (make_short_name) ---
PHRASES_DROP = [
    "de marche", "de course", "de trail", "de randonnée", "de randonnee", "de ville",
    "à lacets", "à scratch -", "à scratch", "en toile", "en cuir", "de trekking", "de basketball", "en daim", "de tennis", "de course", "de marche", "de hockey sur gazon", " à crampon", "de football", "d'haltérophilie", "de tennis multicourt", "de skateboard","de hockey", "intensité moyenne", "respirant garçons", "ultra-respirantes", "/ debutant", "d'entraînement",
]
STOPWORDS_DROP = [
    "chaussure", "chaussures", "basket", "bask'ets", "sneaker", "sneakers",
    "femme", "homme", "enfant", "junior", "garçon", "garcon", "fille", "mixte", "unisex", "unisexe",
    "running", "course", "trail", "randonnée", "randonnee", "fitness", "training",
    "sport", "sportswear", "lifestyle", "marche", "et",
    "modèle", "modele", "impermable", "coloris", "couleur", "color",
    "tailles", "taille", "pointure", "pointures", "imperméable", "résistantes," "résistantes",
    "cassé", "toile", "gomme", "impermeable", "waterproof", "scratch","/", "baskets", "de", "pour", "bébés","bébé", "enfant", "enfants", "moulée", "adolescent","adolescents", "extérieures", "extérieure", "garçons", "filles", "lacets", "femmes", "hommes", "femme", "homme", "kid", "kids", "hombre", "pour", "du", "enfants",
]
COLORS_DROP = [
    "noir", "noire", "noirs", "noires", "black", "blanc", "blanche", "blancs", "blanco", "white",
    "bleu", "bleue", "bleues", "blue", "blueberry", "marine", "navy", "sky", "turquoises", "ultramarine",
    "rouge", "rouges", "red", "bred", "shattered", "rose", "pink", "pinksicle", "pinkscape", "orchid",
    "vert", "verte", "green", "forest", "olive", "kaki", "khaki", "military", "chlorophyll",
    "jaune", "yellow", "mustard", "goldenrod", "gris", "grise", "grises", "grey", "gray",
    "grisport", "silver", "argent", "argenté", "beige", "sand", "camel", "sanddrift",
    "cream", "ivory", "angora", "marron", "brown", "cordura", "hickory", "orange", "corail", "coral",
    "violet", "violette", "violet", "purple", "bordeaux", "aurora", "doré", "jaune", "dore", "jaune", "d'or", "jaune", "gold", "jaune", "golden", "jaune",
    "multicolore", "unique", "multicolores", "unique", "multicolor", "unique", "multi", "unique", "imprimé", "unique",
    "clair", "unique", "foncé", "unique", "foncée", "unique", "pastel", "unique", "uni", "unique"
]
SIZE_PATTERNS = re.compile(
    r"\(\s*\d{2}\s*(?:[-–—]|à|to)\s*\d{2}\s*\)|" +
    r"\b(?:EU|EUR|FR|T(?:aille)?|pointures?)\s*\d{2}\s*(?:[-–—]|à|to)\s*\d{2}\b|" +
    r"\b\d{2}(?:\s*,\s*\d{2}){1,}\b"
)
def make_short_name(text):
    text = str(text).lower()
    for phrase in PHRASES_DROP: text = text.replace(phrase, "")
    sorted_stopwords = sorted(STOPWORDS_DROP + COLORS_DROP, key=len, reverse=True)
    for word in sorted_stopwords: text = re.sub(r'\b' + re.escape(word) + r'\b', '', text)
    text = SIZE_PATTERNS.sub('', text)
    text = re.sub(r'\s+', ' ', text).strip(' -_.,')
    return text if text else str(text)

# --- Fonctions pour la normalisation et la création de hash ---
def to_slug(text):
    text = str(text)
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
    text = text.lower().strip()
    text = re.sub(r'[\s\.\/_]+', '-', text)
    text = re.sub(r'[^\w-]+', '', text)
    return text

def create_sku(text):
    text = str(text)
    return hashlib.sha1(text.encode('utf-8')).hexdigest()[:10]

# --- Dictionnaire de couleurs et fonctions d'extraction ---
COLOR_MAPPING = {
    "noir": "noir", "noire": "noir", "noirs": "noir", "noires": "noir", "black": "noir", "blanc": "blanc", "blanche": "blanc", "blancs": "blanc", "blanco": "blanc", "white": "blanc", "bleu": "bleu", "bleue": "bleu", "bleues": "bleu", "blue": "bleu", "blueberry": "bleu", "marine": "bleu", "navy": "bleu", "sky": "bleu", "turquoises": "bleu", "ultramarine": "bleu", "rouge": "rouge", "rouges": "rouge", "red": "rouge", "bred": "rouge", "shattered": "rouge", "rose": "rose", "pink": "rose", "pinksicle": "rose", "pinkscape": "rose", "orchid": "rose", "vert": "vert", "verte": "vert", "green": "vert", "forest": "vert", "olive": "vert", "kaki": "vert", "khaki": "vert", "military": "vert", "chlorophyll": "vert", "jaune": "jaune", "yellow": "jaune", "mustard": "jaune", "goldenrod": "jaune", "gris": "gris", "grise": "gris", "grises": "gris", "grey": "gris", "gray": "gris", "grisport": "gris", "silver": "gris", "argent": "gris", "argenté": "gris", "beige": "beige", "sand": "beige", "camel": "beige", "sanddrift": "beige", "cream": "beige", "ivory": "beige", "angora": "beige", "marron": "marron", "brown": "marron", "cordura": "marron", "hickory": "marron", "orange": "orange", "corail": "orange", "coral": "orange", "violet": "violet", "violette": "violet", "purple": "violet", "bordeaux": "violet", "aurora": "violet", "doré": "jaune", "dore": "jaune", "d'or": "jaune", "gold": "jaune", "golden": "jaune", "multicolore": "unique", "multicolores": "unique", "multicolor": "unique", "multi": "unique", "imprimé": "unique", "clair": "unique", "foncé": "unique", "foncée": "unique", "pastel": "unique", "uni": "unique"}

def extract_colors_from_text(text: str):
    text_lower = str(text).lower()
    found_colors = []
    BASE_COLORS = [c for c, f in COLOR_MAPPING.items() if f != 'unique']
    MODIFIERS = [c for c, f in COLOR_MAPPING.items() if f == 'unique']
    for color in BASE_COLORS:
        if re.search(r'\b' + re.escape(color) + r'\b', text_lower): found_colors.append(color)
    if found_colors: return found_colors
    for modifier in MODIFIERS:
        if re.search(r'\b' + re.escape(modifier) + r'\b', text_lower): found_colors.append(modifier)
    if found_colors: return found_colors
    return ['unique']

def get_color_info(description):
    colors_found = extract_colors_from_text(description)
    color = colors_found[0]
    color_family = COLOR_MAPPING.get(color, 'unique')
    return color, color_family

# --- Fonctions d'extraction de taille ---
def find_size_in_description(description):
    match = re.search(r'\b(2[89]|[34][0-9])\b', str(description))
    if match: return int(match.group(1))
    return None
size_ranges = {'men': range(39, 49), 'women': range(36, 42), 'child': range(28, 36)}

In [ ]:
# Crée une version brute de la description pour référence
df_concat['model_name_raw'] = df_concat['Description'].astype(str) + ' ' + df_concat['Marque'].astype(str)
df_concat['model_name_raw'] = df_concat['model_name_raw'].apply(lambda x: re.sub(r'\(.*?\)', '', x).strip())

# Utilise la fonction de nettoyage pour créer un nom de modèle PROPRE
df_concat['model_name_clean'] = df_concat['Description'].apply(make_short_name)

# Crée le slug et le SKU de modèle à partir du nom PROPRE
df_concat['model_slug'] = df_concat['model_name_clean'].apply(to_slug)
df_concat['model_slug'] = df_concat['model_slug'].replace('', pd.NA)
df_concat.dropna(subset=['model_slug'], inplace=True)

df_concat['brand_prefix'] = df_concat['Marque'].fillna('').str[:3].str.upper()
df_concat['sku_model_base'] = df_concat['brand_prefix'] + '-' + df_concat['model_slug'].apply(create_sku)

# Regroupe par le slug pour assigner le même sku_model
df_concat['sku_model'] = df_concat.groupby('model_slug')['sku_model_base'].transform('first')
print("✅ OK: SKUs et Slugs de modèle générés avec succès.")

✅ OK: SKUs et Slugs de modèle générés avec succès.


In [ ]:
# Extraction des couleurs
df_concat['color'], df_concat['color_family'] = zip(*df_concat['Description'].apply(get_color_info))

# Création d'un DataFrame vide pour les variants
df_variants = pd.DataFrame()

# Itère sur chaque modèle unique
for sku_model, group in df_concat.groupby('sku_model'):
    color_variants = group.drop_duplicates(subset=['color', 'image_url'])
    for index, row in color_variants.iterrows():
        found_size = find_size_in_description(row['Description'])
        if found_size: sizes_to_use = [found_size]
        else: sizes_to_use = size_ranges.get(row['gender'], [])
        for size in sizes_to_use:
            new_row = row.copy()
            new_row['eu_size'] = size
            new_row['stock_quantity'] = np.random.randint(5, 50)
            variant_data_string = f"{sku_model}-{size}-{row['color']}"
            new_row['sku_variant'] = create_sku(variant_data_string)
            new_row['product_slug'] = row['model_slug']
            df_variants = pd.concat([df_variants, pd.DataFrame([new_row])], ignore_index=True)

df_variants = df_variants.drop_duplicates(subset=['sku_variant'])
print("✅ OK: Les variants ont été générés avec succès.")


✅ OK: Les variants ont été générés avec succès.


In [ ]:
print("-" * 50)
print("✅ Vérification finale des exclusions avant la création des CSV...")

# Vérifier si des marques à exclure sont présentes dans le DataFrame final des variants
brands_in_variants = df_variants['Marque'].unique().tolist()
found_excluded_brands = [brand for brand in brands_in_variants if brand in exclude_brands]

if not found_excluded_brands:
    print("✅ OK : Aucune des marques à exclure n'est présente dans le DataFrame final.")
else:
    print("❌ ERREUR : Les marques suivantes n'ont pas été exclues :", found_excluded_brands)
    # Afficher les lignes problématiques pour le débogage
    print(df_variants[df_variants['Marque'].isin(found_excluded_brands)].head())

# Vérifier si des mots-clés à exclure sont présents dans les descriptions du DataFrame final
found_excluded_keywords = df_variants[df_variants['Description'].str.lower().str.contains('|'.join(exclude_keywords), na=False)]

if found_excluded_keywords.empty:
    print("✅ OK : Aucun des mots-clés à exclure n'est présent dans le DataFrame final.")
else:
    print("❌ ERREUR : Des descriptions avec des mots-clés à exclure n'ont pas été retirées.")
    # Afficher les lignes problématiques
    print(found_excluded_keywords[['Description', 'Marque']].head())

print("-" * 50)

--------------------------------------------------
✅ Vérification finale des exclusions avant la création des CSV...
✅ OK : Aucune des marques à exclure n'est présente dans le DataFrame final.
✅ OK : Aucun des mots-clés à exclure n'est présent dans le DataFrame final.
--------------------------------------------------


In [ ]:
# 1. Vérification de l'unicité des SKUs variants
# Le nombre de lignes doit être égal au nombre de SKUs uniques
if df_variants['sku_variant'].is_unique:
    print("✅ OK : Tous les SKU variants sont uniques.")
else:
    print("❌ ERREUR : Des SKU variants ne sont pas uniques. Vérifiez la logique de création.")
    duplicate_skus = df_variants[df_variants.duplicated(subset=['sku_variant'], keep=False)]
    print("\nExemple de SKUs variants dupliqués :\n", duplicate_skus.head())

print("-" * 50)

# 2. Vérification des valeurs manquantes dans les colonnes clés
missing_values = df_variants[['product_slug', 'eu_size', 'gender', 'price', 'sku_variant', 'sku_model']].isnull().sum()
if missing_values.sum() == 0:
    print("✅ OK : Aucune valeur manquante dans les colonnes clés des variants.")
else:
    print("❌ ERREUR : Des valeurs manquantes ont été trouvées dans les colonnes clés.")
    print(missing_values)

print("-" * 50)

# 3. Vérification de la cohérence des prix
if (df_variants['price'] > 0).all():
    print("✅ OK : Tous les prix sont positifs.")
else:
    print("⚠️ ATTENTION : Des prix non positifs ont été trouvés.")
    print(df_variants[df_variants['price'] <= 0].head())

print("-" * 50)

# 4. Vérification de la liaison avec le modèle
# Chaque variant doit avoir un sku_model correspondant
if not df_variants['sku_model'].isnull().any() and not df_variants['sku_model'].duplicated().all():
    print("✅ OK : La liaison entre les variants et les modèles est correcte.")
else:
    print("❌ ERREUR : Problème de liaison avec le modèle (sku_model manquant ou incorrect).")

print("-" * 50)

# 5. Vérification des tailles extraites
# On peut vérifier la distribution des tailles pour s'assurer qu'elles sont dans les bonnes plages
print("Distribution des tailles par genre :")
print(df_variants.groupby('gender')['eu_size'].describe())

print("-" * 50)

# 6. Vérification de la cohérence entre les slugs et les SKUs
# Les slugs de modèle et les SKUs doivent correspondre
# Pour cette vérification, il faut que 'model_slug' soit présent dans le df_variants
if 'model_slug' in df_variants.columns:
    if (df_variants.groupby('sku_model')['model_slug'].nunique() <= 1).all():
        print("✅ OK : Les slugs de modèle correspondent bien aux SKUs de modèle.")
    else:
        print("❌ ERREUR : Un SKU de modèle est associé à plusieurs slugs de modèle différents.")
        print(df_variants.groupby('sku_model')['model_slug'].nunique())
else:
    print("⚠️ ATTENTION : La colonne 'model_slug' est manquante dans le DataFrame des variants. Recommandation : la conserver pour vérification.")

✅ OK : Tous les SKU variants sont uniques.
--------------------------------------------------
✅ OK : Aucune valeur manquante dans les colonnes clés des variants.
--------------------------------------------------
✅ OK : Tous les prix sont positifs.
--------------------------------------------------
✅ OK : La liaison entre les variants et les modèles est correcte.
--------------------------------------------------
Distribution des tailles par genre :
         count       mean       std   min    25%   50%   75%   max
gender                                                            
child   1890.0  31.497354  2.292355  28.0  29.25  31.0  33.0  35.0
men     9202.0  43.499239  2.872626  39.0  41.00  43.0  46.0  48.0
women   2169.0  38.265560  1.692047  36.0  37.00  38.0  40.0  41.0
--------------------------------------------------
✅ OK : Les slugs de modèle correspondent bien aux SKUs de modèle.


In [ ]:
# --- Étape de vérification et de nettoyage finale ---

# Gérer les couleurs et familles de couleurs vides
# Si 'color' est vide, on le met à 'unique' et 'color_family' est laissé vide.
# Note : la fonction `get_color_info` met déjà 'color' et 'color_family' à 'unique' par défaut.
# Si vous avez des valeurs nulles, cette ligne les gère en plus.
df_variants['color'] = df_variants['color'].fillna('unique')
df_variants['color_family'] = df_variants['color_family'].replace('unique', np.nan) # Remplacer 'unique' par NaN pour le laisser vide

# Vérification des valeurs manquantes dans les colonnes critiques
print("--- Vérification des valeurs manquantes ---")
missing_prices = df_variants[df_variants['price'].isnull()]
if not missing_prices.empty:
    print("❌ ERREUR : Des prix sont manquants dans les variants.")
    print(missing_prices.head())

missing_brands = df_variants[df_variants['Marque'].isnull()]
if not missing_brands.empty:
    print("❌ ERREUR : Des marques sont manquantes dans les variants.")
    print(missing_brands.head())

missing_images = df_variants[df_variants['image_url'].isnull()]
if not missing_images.empty:
    print("❌ ERREUR : Des URLs d'image sont manquantes dans les variants.")
    print(missing_images.head())

if missing_prices.empty and missing_brands.empty and missing_images.empty:
    print("✅ OK : Aucune valeur manquante détectée dans les colonnes clés (prix, marque, image).")


--- Vérification des valeurs manquantes ---
✅ OK : Aucune valeur manquante détectée dans les colonnes clés (prix, marque, image).


In [ ]:
# --- Étape de vérification et de nettoyage finale des variants ---

# Gérer les couleurs et familles de couleurs vides
df_variants['color'] = df_variants['color'].fillna('unique')
df_variants['color_family'] = df_variants['color_family'].replace('unique', np.nan)

# 1. Vérification de l'unicité des SKU variants
if df_variants['sku_variant'].is_unique:
    print("✅ OK : Tous les SKU variants sont uniques.")
else:
    print("❌ ERREUR : Des SKU variants ne sont pas uniques.")
    duplicates = df_variants[df_variants.duplicated(subset=['sku_variant'], keep=False)]
    print("\nExemple de doublons de SKU variant :\n", duplicates[['sku_variant', 'product_slug', 'eu_size']].head())

# 2. Vérification de la cohérence des SKU de modèle et des slugs entre produits et variants
df_products_to_check = df_variants.drop_duplicates(subset=['sku_model']).copy()
df_products_to_check['model_name'] = df_products_to_check['model_name_raw']

# Vérifier la liaison SKU modèle / Slug modèle
sku_model_consistency = df_variants.groupby('sku_model')['model_slug'].nunique()
if (sku_model_consistency == 1).all():
    print("✅ OK : Les SKU de modèle et les slugs de modèle sont cohérents.")
else:
    print("❌ ERREUR : Un SKU de modèle est lié à plusieurs slugs de modèle.")
    print(sku_model_consistency[sku_model_consistency > 1])

# Vérifier la liaison SKU modèle / SKU variant
sku_variant_consistency = df_variants.groupby('sku_model')['sku_variant'].nunique()
if (sku_variant_consistency > 0).all():
    print("✅ OK : Chaque modèle a au moins un SKU de variant lié.")
else:
    print("❌ ERREUR : Certains modèles n'ont pas de SKU de variant lié.")
    print(sku_variant_consistency[sku_variant_consistency == 0])

print("-" * 50)


✅ OK : Tous les SKU variants sont uniques.
✅ OK : Les SKU de modèle et les slugs de modèle sont cohérents.
✅ OK : Chaque modèle a au moins un SKU de variant lié.
--------------------------------------------------


In [ ]:
# 3. Vérification de la validité des URL des images
invalid_urls = df_variants[~df_variants['image_url'].str.startswith('https://contents.mediadecathlon.com/', na=True)]
if invalid_urls.empty:
    print("✅ OK : Toutes les URLs d'image semblent valides (commencent par le préfixe de Decathlon).")
else:
    print("❌ ERREUR : Des URLs d'image non valides ont été trouvées.")
    print("\nExemple d'URLs non valides :\n", invalid_urls[['image_url']].head())

print("-" * 50)


✅ OK : Toutes les URLs d'image semblent valides (commencent par le préfixe de Decathlon).
--------------------------------------------------


In [ ]:
# 1. Création du CSV 'brands'
df_brands = df_variants.drop_duplicates(subset=['Marque']).copy()
df_brands = df_brands.rename(columns={'Marque': 'brand'})
df_brands = df_brands[['brand']]
df_brands.to_csv('brands.csv', index=False)
print("brands.csv créé avec succès.")

brands.csv créé avec succès.


In [ ]:
# 2. Création du CSV 'products'
# On utilise drop_duplicates pour ne garder qu'une ligne par modèle (sku_model)
df_products = df_variants.drop_duplicates(subset=['sku_model']).copy()

df_products = df_products.rename(columns={
    'model_name_clean': 'name',
    'Marque': 'brand',
    'Description': 'description'
})

# On sélectionne les colonnes finales et on les réorganise
df_products = df_products[['model_slug', 'name', 'brand', 'description', 'sku_model']]

# On exporte au format CSV
df_products.to_csv('products.csv', index=False)
print("✅ OK: products.csv créé avec succès.")

✅ OK: products.csv créé avec succès.


In [ ]:
# 3. Création du CSV 'variants'
df_variants_final = df_variants[['product_slug', 'eu_size', 'gender', 'color', 'color_family', 'price', 'stock_quantity', 'sku_variant', 'sku_model', 'image_url']]
df_variants_final.to_csv('variants.csv', index=False)
print("✅ OK: variants.csv créé avec succès.")

✅ OK: variants.csv créé avec succès.


In [ ]:
# 4. Création du CSV 'images'
df_images = df_variants[['sku_variant', 'image_url']].copy()
df_images.to_csv('images.csv', index=False)
print("images.csv créé avec succès.")

images.csv créé avec succès.


In [ ]:
# Assurez-vous d'avoir les DataFrames finaux `df_variants` créés
df_variants_final = df_variants[['product_slug', 'eu_size', 'gender', 'color', 'color_family', 'price', 'stock_quantity', 'sku_variant', 'sku_model', 'image_url']]
df_images = df_variants[['sku_variant', 'image_url']].copy()

# Récupérer les listes de SKUs de chaque DataFrame
skus_in_variants_df = set(df_variants_final['sku_variant'].unique())
skus_in_images_df = set(df_images['sku_variant'].unique())

# Vérifier si les ensembles de SKUs sont identiques
if skus_in_variants_df == skus_in_images_df:
    print("✅ OK: Les SKUs variants dans le CSV `images` sont 100% identiques à ceux du CSV `variants`.")
else:
    # Trouver les SKUs qui ne sont pas dans les deux ensembles
    skus_missing_in_images = skus_in_variants_df - skus_in_images_df
    skus_missing_in_variants = skus_in_images_df - skus_in_variants_df

    if skus_missing_in_images:
        print(f"❌ ERREUR: {len(skus_missing_in_images)} SKUs sont dans `variants` mais absents de `images`.")
        print(f"Exemples de SKUs manquants : {list(skus_missing_in_images)[:5]}")


✅ OK: Les SKUs variants dans le CSV `images` sont 100% identiques à ceux du CSV `variants`.


In [ ]:
# Afficher le nombre total d'entrées dans chaque DataFrame final
print("\n--- Récapitulatif du nombre de lignes dans les fichiers CSV ---")
print(f"Nombre de produits uniques (products.csv) : {df_products['sku_model'].nunique()}")
print(f"Nombre de variants (variants.csv) : {df_variants['sku_variant'].nunique()}")
print(f"Nombre d'images (images.csv) : {df_images['image_url'].nunique()}")

#  Nombre
# print(f"Nombre total de lignes dans df_products : {len(df_products)}")
# print(f"Nombre total de lignes dans df_variants : {len(df_variants)}")
# print(f"Nombre total de lignes dans df_images : {len(df_images)}")


--- Récapitulatif du nombre de lignes dans les fichiers CSV ---
Nombre de produits uniques (products.csv) : 1400
Nombre de variants (variants.csv) : 13261
Nombre d'images (images.csv) : 1578


In [ ]:
# 1. Obtenez le nombre de variantes de base uniques
# On groupe par le sku_model et la couleur pour obtenir un identifiant unique par combinaison
nb_base_variants = df_variants.groupby(['sku_model', 'color']).ngroups

# 2. Obtenez le nombre d'images uniques
nb_images = df_variants['image_url'].nunique()

print(f"Nombre de variantes de base uniques (par modèle et couleur) : {nb_base_variants}")
print(f"Nombre d'images uniques : {nb_images}")



Nombre de variantes de base uniques (par modèle et couleur) : 1462
Nombre d'images uniques : 1578


In [ ]:
# Créez une liste des SKU de modèles qui ont plusieurs couleurs
sku_multi_colors = df_concat.groupby('sku_model')['color'].nunique()
sku_multi_colors = sku_multi_colors[sku_multi_colors > 1].index.tolist()

# Filtrez le DataFrame pour ne garder que ces modèles
df_grouped_models = df_concat[df_concat['sku_model'].isin(sku_multi_colors)]

# Affichez les 10 premiers modèles regroupés avec leurs couleurs et images
print("Voici les 10 premiers modèles qui ont été regroupés :")
for sku, group in df_grouped_models.head(100).groupby('sku_model'):
    print(f"\n--- Modèle : {group['model_name_clean'].iloc[0]} (SKU: {sku}) ---")

    # Affichage des variantes regroupées pour ce modèle
    variants = group[['color', 'image_url', 'Description']].drop_duplicates().to_string(index=False)
    print(variants)

Voici les 10 premiers modèles qui ont été regroupés :

--- Modèle : vl court 3.0 (SKU: ADI-1350a418d1) ---
 color                                                                                                                                                   image_url                        Description
unique   https://contents.mediadecathlon.com/m16561911/k$83ebd48bc3ef50b89f508ae1b7953e89/sq/56d1de55-6b1c-4cb2-b1a2-6a6f93035666_c14c4c14.jpg?format=auto&f=800x0             Chaussure VL Court 3.0
unique https://contents.mediadecathlon.com/m18433678/k$4029b99a6b28afd65184da7464fc673e/sq/6f78f443-febc-4e55-a94c-038797485338_c195c4c158.jpg?format=auto&f=800x0             Chaussure VL Court 3.0
marron                                          https://contents.mediadecathlon.com/p2941240/k$6d363b2af7e2e5d776b06f3c45544dc7/sq/8968397.jpg?format=auto&f=800x0 Basket homme, vl court 3.0, marron
unique   https://contents.mediadecathlon.com/m22155919/k$b97847b4d55ec5267ceda3db44ac0318/sq/8dd07600